Cálculo de métricas de CellPose (benchmark)

In [1]:
# ============================================================
# Calculate_metrics_3D_JGF  — versión con exportación a CSV (RESUMEN SOLO)
# ============================================================

# ---------- CONFIGURACIÓN  ----------
# carpeta EM_Image_Segmentation
code_dir      = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation"
# carpeta mAP_3Dvolume
map_code_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/mAP_3Dvolume"

# carpeta con .tif/.tiff de predicción (Cellpose/PlantSeg/BiaPy...)
# CellPose (benchmark):
input_dir = "C:/Users/Usuario/Desktop/TFM/TFMv2/Results CellPose -- SAM - actualizado"

# carpeta con las GT (mismo nombre que la predicción; si predicción acaba en .tiff, GT puede ser .tif)
gt_dir        = "C:/Users/Usuario/Desktop/TFM/TFMv2/ResUnet 3D GoldStandard - BiaPy/Test/y"
# carpeta temporal donde se generarán ficheros h5/txt
temporal_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation/utils/scripts/temp"
# ----------------------------------------------------------

# ---------- OPCIONES DE CÁLCULO ----------
iou                 = True   # calcular IoU y VOC
mAP                 = True   # calcular mAP@0.50 y mAP@0.75
matching_stats      = True   # matching simple (DatasetMatching)
matching_segcompare = True   # matching con segCompare (si está disponible)
matching_stats_ths  = (0.3, 0.5, 0.75)  # umbrales para resumen de matching
verbose             = True
# -----------------------------------------

# ============ IMPORTS BÁSICOS ============
import os, sys, re
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.io import imread
import h5py
from dataclasses import asdict, is_dataclass

# Preparar paths
os.makedirs(temporal_dir, exist_ok=True)
gt_partial_files_dir = os.path.join(temporal_dir, "GT")
os.makedirs(gt_partial_files_dir, exist_ok=True)

# Para poder importar utilidades del repo
sys.path.insert(0, code_dir)
sys.path.insert(0, map_code_dir)

# === Imports de tus utilidades ===
# EM_Image_Segmentation
from utils.matching import matching, match_using_segCompare
from utils.util import save_tif_pair_discard, wrapper_matching_dataset_lazy, wrapper_matching_segCompare
from engine.metrics import jaccard_index_numpy, voc_calculation

# mAP_3Dvolume
from demo_modified import main as mAP_calculation

# ===== Helpers para extraer métricas de DatasetMatching =====
def _datasetmatching_to_dict(dm):
    """
    Intenta leer atributos (precision, recall, accuracy, f1, panoptic_quality,
    mean_true_score, mean_matched_score, thresh). Si no existen (p.ej. es una
    clase distinta), hace un parseo robusto del str(dm).
    """
    keys = ["precision","recall","accuracy","f1","panoptic_quality",
            "mean_true_score","mean_matched_score","thresh"]
    out = {k: None for k in keys}

    # 1) intento directo por atributos
    for k in keys:
        if hasattr(dm, k):
            try:
                out[k] = float(getattr(dm, k))
            except Exception:
                pass

    # 2) si faltan, intento por dataclass
    if any(v is None for v in out.values()) and is_dataclass(dm):
        try:
            d = asdict(dm)
            for k in keys:
                if k in d and out[k] is None:
                    out[k] = float(d[k])
        except Exception:
            pass

    # 3) parseo de str como último recurso
    if any(v is None for v in out.values()):
        s = str(dm)
        for k in keys:
            if out[k] is None:
                m = re.search(rf"{k}\s*=\s*([0-9]*\.?[0-9]+)", s)
                if m:
                    out[k] = float(m.group(1))

    return out

# ============ ACUMULADORES Y CSV ============
run_name   = Path(input_dir.rstrip("/\\")).name
csv_out_dir = Path(temporal_dir) / f"metrics_{run_name}"
csv_out_dir.mkdir(parents=True, exist_ok=True)

iou_fg_list = []
iou_overall_list = []
mAP_50_total = 0.0
mAP_75_total = 0.0
all_matching_stats = []
all_matching_stats_segCompare = []

# ============ LISTADO DE IMÁGENES ============
ids = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.tif', '.tiff'))])
if len(ids) == 0:
    raise RuntimeError(f"No se han encontrado .tif/.tiff en: {input_dir}")

# ============ BUCLE PRINCIPAL POR IMAGEN ============
for id_ in ids:
    # Rutas de predicción y GT
    img_path = os.path.join(input_dir, id_)

    base_noext = os.path.splitext(id_)[0]
    # Si la predicción es .tiff, probamos GT .tif
    if id_.lower().endswith(".tiff"):
        mask_id = base_noext + ".tif"
    else:
        mask_id = id_

    mask_path = os.path.join(gt_dir, mask_id)
    if not os.path.isfile(mask_path):
        # Intenta la otra extensión por si acaso
        alt = base_noext + (".tiff" if id_.lower().endswith(".tif") else ".tif")
        alt_path = os.path.join(gt_dir, alt)
        if os.path.isfile(alt_path):
            mask_path = alt_path
        else:
            print(f"[AVISO] No existe GT para {id_}. Se omite esta imagen.")
            continue

    # Cargar predicción y GT
    img  = imread(img_path).astype(np.int64)
    mask = imread(mask_path).astype(np.int64)

    if verbose:
        print(f"\n### Analizando: {id_}")

    # Carpeta temporal por imagen
    file_dir = os.path.join(temporal_dir, base_noext)
    os.makedirs(file_dir, exist_ok=True)

    # -------- IoU / VOC --------
    if iou:
        _iou    = jaccard_index_numpy((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8))
        _ov_iou = voc_calculation((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8), _iou)
        iou_fg_list.append(float(_iou))
        iou_overall_list.append(float(_ov_iou))
        if verbose:
            print(f"Foreground IoU: { _iou }  -- Overall IoU: { _ov_iou }")

    # -------- mAP (vía mAP_3Dvolume) --------
    if mAP:
        # 1) H5 de predicción
        h5file_name = os.path.join(file_dir, base_noext + ".h5")
        if os.path.exists(h5file_name):
            try:
                os.remove(h5file_name)
            except PermissionError:
                pass
        with h5py.File(h5file_name, "w") as h5f:
            h5f.create_dataset("dataset", data=img, compression="lzf")

        # 2) H5 de GT (si no existe ya)
        gt_f = os.path.join(gt_partial_files_dir, base_noext + ".h5")
        if not os.path.isfile(gt_f):
            with h5py.File(gt_f, "w") as h5f:
                h5f.create_dataset("dataset", data=mask.squeeze(), compression="lzf")

        # 3) Scores falsos (etiquetas únicas != 0)
        labels = np.unique(img)
        labels = labels[labels != 0].astype(int)

        if labels.size == 0:
            if verbose:
                print(f"[mAP] {id_} no tiene instancias segmentadas. Se omite mAP.")
        else:
            scores = np.ones_like(labels, dtype=np.float32)
            df_scores = pd.DataFrame(np.stack([labels, scores], axis=1))
            score_path = os.path.splitext(h5file_name)[0] + "_scores.txt"
            df_scores.to_csv(score_path, sep="\t", index=False, header=False)

            # --- construir argumentos correctos para mAP_3Dvolume ---
            from types import SimpleNamespace as Namespace
            args = Namespace(
                gt_seg=gt_f,
                predict_seg=h5file_name,
                predict_score=score_path,
                threshold="5e-3, 3e-4",
                threshold_crumb_64=64,   # compat
                threshold_crumb=64,      # el que usa demo_modified.py
                chunk_size=250,
                output_name=file_dir,
                do_txt=1,
                do_eval=1,
                slices="-1",
            )
            mAP_calculation(args)

            # 5) Leer resultados desde el txt que genera mAP_3Dvolume
            map_txt = os.path.join(file_dir, "nuclu_mAP.txt")
            if os.path.isfile(map_txt):
                with open(map_txt, "r") as f:
                    for line in f:
                        if "mAP @ IoU 0.50" in line:
                            try: mAP_50_total += float(line.split()[-1])
                            except: pass
                        if "mAP @ IoU 0.75" in line:
                            try: mAP_75_total += float(line.split()[-1])
                            except: pass

    # -------- Matching por imagen --------
    if matching_stats:
        try:
            r_stats = matching(mask, img, thresh=matching_stats_ths, report_matches=False)
            all_matching_stats.append(r_stats)
        except Exception as e:
            print(f"[Matching] Error en {id_}: {e}")

    # -------- Matching segCompare por imagen (opcional) --------
    if matching_segcompare:
        if np.max(mask) == 0:
            if verbose: print("GT vacía; se omite segCompare.")
        else:
            try:
                r_stats_segCompare = match_using_segCompare(mask, img)
                all_matching_stats_segCompare.append(r_stats_segCompare)
            except ZeroDivisionError:
                print(f"[segCompare] División por cero en {id_}; se omite.")
            except Exception as e:
                print(f"[segCompare] Error en {id_}: {e}")

# ============ RESUMEN GLOBAL / WRAPPERS ============
print("\n######### RESULTADOS (RESUMEN) #########")

# IoU promedio
iou_fg_mean = float(np.nanmean(iou_fg_list)) if len(iou_fg_list) else None
iou_ov_mean = float(np.nanmean(iou_overall_list)) if len(iou_overall_list) else None

if iou:
    print(f"Foreground IoU medio: {iou_fg_mean}")
    print(f"Overall IoU medio:    {iou_ov_mean}")

# Matching dataset (wrapper)
stats_dataset = None
if matching_stats and len(all_matching_stats) > 0:
    stats_dataset = wrapper_matching_dataset_lazy(all_matching_stats, matching_stats_ths)
    print("Resumen Matching stats (dataset):", stats_dataset)

# Matching segCompare dataset (wrapper)
stats_segCompare_dataset = None
if matching_segcompare and len(all_matching_stats_segCompare) > 0:
    stats_segCompare_dataset = wrapper_matching_segCompare(all_matching_stats_segCompare)
    print("Resumen Matching segCompare (dataset):", stats_segCompare_dataset)

# ============ CREAR CSV **SOLO RESUMEN** ============
# Elegimos, si existe, el DatasetMatching del umbral 0.5
dm_pick = {}
if isinstance(stats_dataset, (list, tuple)) and len(stats_dataset) > 0:
    # intentar coger thresh=0.5
    for dm in stats_dataset:
        d = _datasetmatching_to_dict(dm)
        if d.get("thresh") is not None and abs(d["thresh"] - 0.5) < 1e-6:
            dm_pick = d
            break
    if not dm_pick:
        dm_pick = _datasetmatching_to_dict(stats_dataset[0])

# campos del segCompare 
segc = stats_segCompare_dataset if isinstance(stats_segCompare_dataset, dict) else {}

summary = {
    "foreground_iou_mean": iou_fg_mean,
    "overall_iou_mean":    iou_ov_mean,
    "AP@0.50":             float(mAP_50_total) if mAP else None,
    "AP@0.75":             float(mAP_75_total) if mAP else None,
    "precision":           dm_pick.get("precision"),
    "recall":              dm_pick.get("recall"),
    "accuracy":            dm_pick.get("accuracy"),
    "f1":                  dm_pick.get("f1"),
    "panoptic_quality":    dm_pick.get("panoptic_quality"),
    "mean_true_score":     dm_pick.get("mean_true_score"),
    "mean_matched_score":  dm_pick.get("mean_matched_score"),
    "n_cells":                 float(segc.get("number_of_cells", np.nan)) if segc else None,
    "correct_segmentations":   float(segc.get("correct_segmentations", np.nan)) if segc else None,
    "oversegmentation_rate":   float(segc.get("oversegmentation_rate", np.nan)) if segc else None,
    "undersegmentation_rate":  float(segc.get("undersegmentation_rate", np.nan)) if segc else None,
    "missing_rate":            float(segc.get("missing_rate", np.nan)) if segc else None,
}

# orden de columnas
cols_order = [
    "foreground_iou_mean","overall_iou_mean",
    "AP@0.50","AP@0.75",
    "precision","recall","accuracy","f1","panoptic_quality",
    "mean_true_score","mean_matched_score",
    "n_cells","correct_segmentations",
    "oversegmentation_rate","undersegmentation_rate","missing_rate"
]

df_summary = pd.DataFrame([summary])
df_summary = df_summary[[c for c in cols_order if c in df_summary.columns]]

csv_path = csv_out_dir / f"metrics_{run_name}.csv"
df_summary.to_csv(csv_path, index=False)
print(f"\nCSV RESUMEN guardado en:\n{csv_path}")


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_4.tif
Foreground IoU: 0.8492197103308214  -- Overall IoU: 0.8756194208007926
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


54it [00:00, 79.45it/s] 


	-RUNTIME:	2.382 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.560
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.895
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.749
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_5.tif
Foreground IoU: 0.9033106597480537  -- Overall IoU: 0.9168864002182187
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


48it [00:00, 73.68it/s]


	-RUNTIME:	2.049 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.921
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.921
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series002_7.tif
Foreground IoU: 0.908638674623103  -- Overall IoU: 0.9256013144357593
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


...

	-RUNTIME:	1.857 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.751
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.630
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240920_sdk_Dlg568001-Series001_8.tif
Foreground IoU: 0.8946905925606802  -- Overall IoU: 0.9217297422417374
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


63it [00:00, 153.91it/s]


	-RUNTIME:	1.795 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.563
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.897
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.786
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240920_sdk_Dlg568001-Series002_10.tif
Foreground IoU: 0.8681083115918918  -- Overall IoU: 0.9097568985736925
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


66it [00:00, 113.53it/s]


	-RUNTIME:	2.008 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.577
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.939
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.779
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


######### RESULTADOS (RESUMEN) #########
Foreground IoU medio: 0.8469963605315967
Overall IoU medio:    0.8885360540197402
Resumen Matching stats (dataset): (DatasetMatching(criterion='iou', thresh=0.3, fp=3308, tp=868, fn=127, precision=0.2078544061302682, recall=0.8723618090452261, accuracy=0.20171973042063676, f1=0.33571842970411914, n_true=995, n_pred=4176, mean_true_score=0.6847327563031834, mean_matched_score=0.7849183093567598, panoptic_quality=0.2635115422632634, by_image=False), DatasetMatching(criterion='iou', thresh=0.5, fp=3368, tp=808, fn=187, precision=0.19348659003

In [ ]:
Cálculo métricas BiaPy + PlantSeg (modelo real corregido)

In [1]:
# ============================================================
# Calculate_metrics_3D_JGF  — versión con exportación a CSV (RESUMEN SOLO)
# ============================================================

# ---------- CONFIGURACIÓN  ----------
# carpeta EM_Image_Segmentation
code_dir      = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation"
# carpeta mAP_3Dvolume
map_code_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/mAP_3Dvolume"

# carpeta con .tif/.tiff de predicción (Cellpose/PlantSeg/BiaPy...)
# BiaPy + PlantSeg (modelo real corregido):
input_dir = "C:/Users/Usuario/Desktop/TFM/TFMv2/AllTrainings/embryosMiniCrops_New/results/PlantSegInstances"

# carpeta con las GT (mismo nombre que la predicción; si predicción acaba en .tiff, GT puede ser .tif)
gt_dir        = "C:/Users/Usuario/Desktop/TFM/TFMv2/ResUnet 3D GoldStandard - BiaPy/Test/y"
# carpeta temporal donde se generarán ficheros h5/txt
temporal_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation/utils/scripts/temp"
# ----------------------------------------------------------

# ---------- OPCIONES DE CÁLCULO ----------
iou                 = True   # calcular IoU y VOC
mAP                 = True   # calcular mAP@0.50 y mAP@0.75
matching_stats      = True   # matching simple (DatasetMatching)
matching_segcompare = True   # matching con segCompare (si está disponible)
matching_stats_ths  = (0.3, 0.5, 0.75)  # umbrales para resumen de matching
verbose             = True
# -----------------------------------------

# ============ IMPORTS BÁSICOS ============
import os, sys, re
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.io import imread
import h5py
from dataclasses import asdict, is_dataclass

# Preparar paths
os.makedirs(temporal_dir, exist_ok=True)
gt_partial_files_dir = os.path.join(temporal_dir, "GT")
os.makedirs(gt_partial_files_dir, exist_ok=True)

# Para poder importar utilidades del repo
sys.path.insert(0, code_dir)
sys.path.insert(0, map_code_dir)

# === Imports de tus utilidades ===
# EM_Image_Segmentation
from utils.matching import matching, match_using_segCompare
from utils.util import save_tif_pair_discard, wrapper_matching_dataset_lazy, wrapper_matching_segCompare
from engine.metrics import jaccard_index_numpy, voc_calculation

# mAP_3Dvolume
from demo_modified import main as mAP_calculation

# ===== Helpers para extraer métricas de DatasetMatching =====
def _datasetmatching_to_dict(dm):
    """
    Intenta leer atributos (precision, recall, accuracy, f1, panoptic_quality,
    mean_true_score, mean_matched_score, thresh). Si no existen (p.ej. es una
    clase distinta), hace un parseo robusto del str(dm).
    """
    keys = ["precision","recall","accuracy","f1","panoptic_quality",
            "mean_true_score","mean_matched_score","thresh"]
    out = {k: None for k in keys}

    # 1) intento directo por atributos
    for k in keys:
        if hasattr(dm, k):
            try:
                out[k] = float(getattr(dm, k))
            except Exception:
                pass

    # 2) si faltan, intento por dataclass
    if any(v is None for v in out.values()) and is_dataclass(dm):
        try:
            d = asdict(dm)
            for k in keys:
                if k in d and out[k] is None:
                    out[k] = float(d[k])
        except Exception:
            pass

    # 3) parseo de str como último recurso
    if any(v is None for v in out.values()):
        s = str(dm)
        for k in keys:
            if out[k] is None:
                m = re.search(rf"{k}\s*=\s*([0-9]*\.?[0-9]+)", s)
                if m:
                    out[k] = float(m.group(1))

    return out

# ============ ACUMULADORES Y CSV ============
run_name   = Path(input_dir.rstrip("/\\")).name
csv_out_dir = Path(temporal_dir) / f"metrics_{run_name}"
csv_out_dir.mkdir(parents=True, exist_ok=True)

iou_fg_list = []
iou_overall_list = []
mAP_50_total = 0.0
mAP_75_total = 0.0
all_matching_stats = []
all_matching_stats_segCompare = []

# ============ LISTADO DE IMÁGENES ============
ids = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.tif', '.tiff'))])
if len(ids) == 0:
    raise RuntimeError(f"No se han encontrado .tif/.tiff en: {input_dir}")

# ============ BUCLE PRINCIPAL POR IMAGEN ============
for id_ in ids:
    # Rutas de predicción y GT
    img_path = os.path.join(input_dir, id_)

    base_noext = os.path.splitext(id_)[0]
    # Si la predicción es .tiff, probamos GT .tif
    if id_.lower().endswith(".tiff"):
        mask_id = base_noext + ".tif"
    else:
        mask_id = id_

    mask_path = os.path.join(gt_dir, mask_id)
    if not os.path.isfile(mask_path):
        # Intenta la otra extensión por si acaso
        alt = base_noext + (".tiff" if id_.lower().endswith(".tif") else ".tif")
        alt_path = os.path.join(gt_dir, alt)
        if os.path.isfile(alt_path):
            mask_path = alt_path
        else:
            print(f"[AVISO] No existe GT para {id_}. Se omite esta imagen.")
            continue

    # Cargar predicción y GT
    img  = imread(img_path).astype(np.int64)
    mask = imread(mask_path).astype(np.int64)

    if verbose:
        print(f"\n### Analizando: {id_}")

    # Carpeta temporal por imagen
    file_dir = os.path.join(temporal_dir, base_noext)
    os.makedirs(file_dir, exist_ok=True)

    # -------- IoU / VOC --------
    if iou:
        _iou    = jaccard_index_numpy((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8))
        _ov_iou = voc_calculation((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8), _iou)
        iou_fg_list.append(float(_iou))
        iou_overall_list.append(float(_ov_iou))
        if verbose:
            print(f"Foreground IoU: { _iou }  -- Overall IoU: { _ov_iou }")

    # -------- mAP (vía mAP_3Dvolume) --------
    if mAP:
        # 1) H5 de predicción
        h5file_name = os.path.join(file_dir, base_noext + ".h5")
        if os.path.exists(h5file_name):
            try:
                os.remove(h5file_name)
            except PermissionError:
                pass
        with h5py.File(h5file_name, "w") as h5f:
            h5f.create_dataset("dataset", data=img, compression="lzf")

        # 2) H5 de GT (si no existe ya)
        gt_f = os.path.join(gt_partial_files_dir, base_noext + ".h5")
        if not os.path.isfile(gt_f):
            with h5py.File(gt_f, "w") as h5f:
                h5f.create_dataset("dataset", data=mask.squeeze(), compression="lzf")

        # 3) Scores falsos (etiquetas únicas != 0)
        labels = np.unique(img)
        labels = labels[labels != 0].astype(int)

        if labels.size == 0:
            if verbose:
                print(f"[mAP] {id_} no tiene instancias segmentadas. Se omite mAP.")
        else:
            scores = np.ones_like(labels, dtype=np.float32)
            df_scores = pd.DataFrame(np.stack([labels, scores], axis=1))
            score_path = os.path.splitext(h5file_name)[0] + "_scores.txt"
            df_scores.to_csv(score_path, sep="\t", index=False, header=False)

            # --- construir argumentos correctos para mAP_3Dvolume ---
            from types import SimpleNamespace as Namespace
            args = Namespace(
                gt_seg=gt_f,
                predict_seg=h5file_name,
                predict_score=score_path,
                threshold="5e-3, 3e-4",
                threshold_crumb_64=64,   # compat
                threshold_crumb=64,      # el que usa demo_modified.py
                chunk_size=250,
                output_name=file_dir,
                do_txt=1,
                do_eval=1,
                slices="-1",
            )
            mAP_calculation(args)

            # 5) Leer resultados desde el txt que genera mAP_3Dvolume
            map_txt = os.path.join(file_dir, "nuclu_mAP.txt")
            if os.path.isfile(map_txt):
                with open(map_txt, "r") as f:
                    for line in f:
                        if "mAP @ IoU 0.50" in line:
                            try: mAP_50_total += float(line.split()[-1])
                            except: pass
                        if "mAP @ IoU 0.75" in line:
                            try: mAP_75_total += float(line.split()[-1])
                            except: pass

    # -------- Matching por imagen --------
    if matching_stats:
        try:
            r_stats = matching(mask, img, thresh=matching_stats_ths, report_matches=False)
            all_matching_stats.append(r_stats)
        except Exception as e:
            print(f"[Matching] Error en {id_}: {e}")

    # -------- Matching segCompare por imagen (opcional) --------
    if matching_segcompare:
        if np.max(mask) == 0:
            if verbose: print("GT vacía; se omite segCompare.")
        else:
            try:
                r_stats_segCompare = match_using_segCompare(mask, img)
                all_matching_stats_segCompare.append(r_stats_segCompare)
            except ZeroDivisionError:
                print(f"[segCompare] División por cero en {id_}; se omite.")
            except Exception as e:
                print(f"[segCompare] Error en {id_}: {e}")

# ============ RESUMEN GLOBAL / WRAPPERS ============
print("\n######### RESULTADOS (RESUMEN) #########")

# IoU promedio
iou_fg_mean = float(np.nanmean(iou_fg_list)) if len(iou_fg_list) else None
iou_ov_mean = float(np.nanmean(iou_overall_list)) if len(iou_overall_list) else None

if iou:
    print(f"Foreground IoU medio: {iou_fg_mean}")
    print(f"Overall IoU medio:    {iou_ov_mean}")

# Matching dataset (wrapper)
stats_dataset = None
if matching_stats and len(all_matching_stats) > 0:
    stats_dataset = wrapper_matching_dataset_lazy(all_matching_stats, matching_stats_ths)
    print("Resumen Matching stats (dataset):", stats_dataset)

# Matching segCompare dataset (wrapper)
stats_segCompare_dataset = None
if matching_segcompare and len(all_matching_stats_segCompare) > 0:
    stats_segCompare_dataset = wrapper_matching_segCompare(all_matching_stats_segCompare)
    print("Resumen Matching segCompare (dataset):", stats_segCompare_dataset)

# ============ CREAR CSV **SOLO RESUMEN** ============
# Elegimos, si existe, el DatasetMatching del umbral 0.5
dm_pick = {}
if isinstance(stats_dataset, (list, tuple)) and len(stats_dataset) > 0:
    # intentar coger thresh=0.5
    for dm in stats_dataset:
        d = _datasetmatching_to_dict(dm)
        if d.get("thresh") is not None and abs(d["thresh"] - 0.5) < 1e-6:
            dm_pick = d
            break
    if not dm_pick:
        dm_pick = _datasetmatching_to_dict(stats_dataset[0])

# campos del segCompare 
segc = stats_segCompare_dataset if isinstance(stats_segCompare_dataset, dict) else {}

summary = {
    "foreground_iou_mean": iou_fg_mean,
    "overall_iou_mean":    iou_ov_mean,
    "AP@0.50":             float(mAP_50_total) if mAP else None,
    "AP@0.75":             float(mAP_75_total) if mAP else None,
    "precision":           dm_pick.get("precision"),
    "recall":              dm_pick.get("recall"),
    "accuracy":            dm_pick.get("accuracy"),
    "f1":                  dm_pick.get("f1"),
    "panoptic_quality":    dm_pick.get("panoptic_quality"),
    "mean_true_score":     dm_pick.get("mean_true_score"),
    "mean_matched_score":  dm_pick.get("mean_matched_score"),
    "n_cells":                 float(segc.get("number_of_cells", np.nan)) if segc else None,
    "correct_segmentations":   float(segc.get("correct_segmentations", np.nan)) if segc else None,
    "oversegmentation_rate":   float(segc.get("oversegmentation_rate", np.nan)) if segc else None,
    "undersegmentation_rate":  float(segc.get("undersegmentation_rate", np.nan)) if segc else None,
    "missing_rate":            float(segc.get("missing_rate", np.nan)) if segc else None,
}

# orden de columnas
cols_order = [
    "foreground_iou_mean","overall_iou_mean",
    "AP@0.50","AP@0.75",
    "precision","recall","accuracy","f1","panoptic_quality",
    "mean_true_score","mean_matched_score",
    "n_cells","correct_segmentations",
    "oversegmentation_rate","undersegmentation_rate","missing_rate"
]

df_summary = pd.DataFrame([summary])
df_summary = df_summary[[c for c in cols_order if c in df_summary.columns]]

csv_path = csv_out_dir / f"metrics_{run_name}.csv"
df_summary.to_csv(csv_path, index=False)
print(f"\nCSV RESUMEN guardado en:\n{csv_path}")


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_4.tiff
Foreground IoU: 0.8768320529815604  -- Overall IoU: 0.8991615166062212
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


51it [00:01, 38.34it/s]


	-RUNTIME:	3.75 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.683
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.960
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.900
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_5.tiff
Foreground IoU: 0.931144038618833  -- Overall IoU: 0.9413840212209991
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


52it [00:00, 71.02it/s]


	-RUNTIME:	3.046 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.734
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.980
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.900
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.143


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series002_7.tiff
Foreground IoU: 0.9388546540689814  -- Overall IoU: 0.9504623772760084
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


...

	-RUNTIME:	2.991 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.730
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.971
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.923
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.007


### Analizando: 20240920_sdk_Dlg568001-Series001_8.tiff
Foreground IoU: 0.9042456461928332  -- Overall IoU: 0.9288858448674302
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


62it [00:00, 86.92it/s] 


	-RUNTIME:	2.756 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.610
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.817
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240920_sdk_Dlg568001-Series002_10.tiff
Foreground IoU: 0.8983171264101187  -- Overall IoU: 0.930731785050692
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


65it [00:01, 63.93it/s] 


	-RUNTIME:	3.73 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.660
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.950
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.862
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.002


######### RESULTADOS (RESUMEN) #########
Foreground IoU medio: 0.9120650835017169
Overall IoU medio:    0.9333824098644619
Resumen Matching stats (dataset): (DatasetMatching(criterion='iou', thresh=0.3, fp=9, tp=961, fn=34, precision=0.9907216494845361, recall=0.9658291457286432, accuracy=0.9571713147410359, f1=0.978117048346056, n_true=995, n_pred=970, mean_true_score=0.8133830928323257, mean_matched_score=0.842160434306102, panoptic_quality=0.8237314782373171, by_image=False), DatasetMatching(criterion='iou', thresh=0.5, fp=14, tp=956, fn=39, precision=0.9855670103092784, recall

In [ ]:
Cálculo métricas BiaPy + PlantSeg con fake embryos (GANs)

In [1]:
# ============================================================
# Calculate_metrics_3D_JGF  — versión con exportación a CSV (RESUMEN SOLO)
# ============================================================

# ---------- CONFIGURACIÓN  ----------
# carpeta EM_Image_Segmentation
code_dir      = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation"
# carpeta mAP_3Dvolume
map_code_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/mAP_3Dvolume"

# carpeta con .tif/.tiff de predicción (Cellpose/PlantSeg/BiaPy...)
# BiaPy + PlantSeg con fake embryos (GANs):
input_dir = "C:/Users/Usuario/Desktop/TFM/TFMv2/AllTrainings/fakeEmbryosTraining/PlantSegInstances"

# carpeta con las GT (mismo nombre que la predicción; si predicción acaba en .tiff, GT puede ser .tif)
gt_dir        = "C:/Users/Usuario/Desktop/TFM/TFMv2/ResUnet 3D GoldStandard - BiaPy/Test/y"
# carpeta temporal donde se generarán ficheros h5/txt
temporal_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation/utils/scripts/temp"
# ----------------------------------------------------------

# ---------- OPCIONES DE CÁLCULO ----------
iou                 = True   # calcular IoU y VOC
mAP                 = True   # calcular mAP@0.50 y mAP@0.75
matching_stats      = True   # matching simple (DatasetMatching)
matching_segcompare = True   # matching con segCompare (si está disponible)
matching_stats_ths  = (0.3, 0.5, 0.75)  # umbrales para resumen de matching
verbose             = True
# -----------------------------------------

# ============ IMPORTS BÁSICOS ============
import os, sys, re
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.io import imread
import h5py
from dataclasses import asdict, is_dataclass

# Preparar paths
os.makedirs(temporal_dir, exist_ok=True)
gt_partial_files_dir = os.path.join(temporal_dir, "GT")
os.makedirs(gt_partial_files_dir, exist_ok=True)

# Para poder importar utilidades del repo
sys.path.insert(0, code_dir)
sys.path.insert(0, map_code_dir)

# === Imports de tus utilidades ===
# EM_Image_Segmentation
from utils.matching import matching, match_using_segCompare
from utils.util import save_tif_pair_discard, wrapper_matching_dataset_lazy, wrapper_matching_segCompare
from engine.metrics import jaccard_index_numpy, voc_calculation

# mAP_3Dvolume
from demo_modified import main as mAP_calculation

# ===== Helpers para extraer métricas de DatasetMatching =====
def _datasetmatching_to_dict(dm):
    """
    Intenta leer atributos (precision, recall, accuracy, f1, panoptic_quality,
    mean_true_score, mean_matched_score, thresh). Si no existen (p.ej. es una
    clase distinta), hace un parseo robusto del str(dm).
    """
    keys = ["precision","recall","accuracy","f1","panoptic_quality",
            "mean_true_score","mean_matched_score","thresh"]
    out = {k: None for k in keys}

    # 1) intento directo por atributos
    for k in keys:
        if hasattr(dm, k):
            try:
                out[k] = float(getattr(dm, k))
            except Exception:
                pass

    # 2) si faltan, intento por dataclass
    if any(v is None for v in out.values()) and is_dataclass(dm):
        try:
            d = asdict(dm)
            for k in keys:
                if k in d and out[k] is None:
                    out[k] = float(d[k])
        except Exception:
            pass

    # 3) parseo de str como último recurso
    if any(v is None for v in out.values()):
        s = str(dm)
        for k in keys:
            if out[k] is None:
                m = re.search(rf"{k}\s*=\s*([0-9]*\.?[0-9]+)", s)
                if m:
                    out[k] = float(m.group(1))

    return out

# ============ ACUMULADORES Y CSV ============
run_name   = Path(input_dir.rstrip("/\\")).name
csv_out_dir = Path(temporal_dir) / f"metrics_{run_name}"
csv_out_dir.mkdir(parents=True, exist_ok=True)

iou_fg_list = []
iou_overall_list = []
mAP_50_total = 0.0
mAP_75_total = 0.0
all_matching_stats = []
all_matching_stats_segCompare = []

# ============ LISTADO DE IMÁGENES ============
ids = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.tif', '.tiff'))])
if len(ids) == 0:
    raise RuntimeError(f"No se han encontrado .tif/.tiff en: {input_dir}")

# ============ BUCLE PRINCIPAL POR IMAGEN ============
for id_ in ids:
    # Rutas de predicción y GT
    img_path = os.path.join(input_dir, id_)

    base_noext = os.path.splitext(id_)[0]
    # Si la predicción es .tiff, probamos GT .tif
    if id_.lower().endswith(".tiff"):
        mask_id = base_noext + ".tif"
    else:
        mask_id = id_

    mask_path = os.path.join(gt_dir, mask_id)
    if not os.path.isfile(mask_path):
        # Intenta la otra extensión por si acaso
        alt = base_noext + (".tiff" if id_.lower().endswith(".tif") else ".tif")
        alt_path = os.path.join(gt_dir, alt)
        if os.path.isfile(alt_path):
            mask_path = alt_path
        else:
            print(f"[AVISO] No existe GT para {id_}. Se omite esta imagen.")
            continue

    # Cargar predicción y GT
    img  = imread(img_path).astype(np.int64)
    mask = imread(mask_path).astype(np.int64)

    if verbose:
        print(f"\n### Analizando: {id_}")

    # Carpeta temporal por imagen
    file_dir = os.path.join(temporal_dir, base_noext)
    os.makedirs(file_dir, exist_ok=True)

    # -------- IoU / VOC --------
    if iou:
        _iou    = jaccard_index_numpy((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8))
        _ov_iou = voc_calculation((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8), _iou)
        iou_fg_list.append(float(_iou))
        iou_overall_list.append(float(_ov_iou))
        if verbose:
            print(f"Foreground IoU: { _iou }  -- Overall IoU: { _ov_iou }")

    # -------- mAP (vía mAP_3Dvolume) --------
    if mAP:
        # 1) H5 de predicción
        h5file_name = os.path.join(file_dir, base_noext + ".h5")
        if os.path.exists(h5file_name):
            try:
                os.remove(h5file_name)
            except PermissionError:
                pass
        with h5py.File(h5file_name, "w") as h5f:
            h5f.create_dataset("dataset", data=img, compression="lzf")

        # 2) H5 de GT (si no existe ya)
        gt_f = os.path.join(gt_partial_files_dir, base_noext + ".h5")
        if not os.path.isfile(gt_f):
            with h5py.File(gt_f, "w") as h5f:
                h5f.create_dataset("dataset", data=mask.squeeze(), compression="lzf")

        # 3) Scores falsos (etiquetas únicas != 0)
        labels = np.unique(img)
        labels = labels[labels != 0].astype(int)

        if labels.size == 0:
            if verbose:
                print(f"[mAP] {id_} no tiene instancias segmentadas. Se omite mAP.")
        else:
            scores = np.ones_like(labels, dtype=np.float32)
            df_scores = pd.DataFrame(np.stack([labels, scores], axis=1))
            score_path = os.path.splitext(h5file_name)[0] + "_scores.txt"
            df_scores.to_csv(score_path, sep="\t", index=False, header=False)

            # --- construir argumentos correctos para mAP_3Dvolume ---
            from types import SimpleNamespace as Namespace
            args = Namespace(
                gt_seg=gt_f,
                predict_seg=h5file_name,
                predict_score=score_path,
                threshold="5e-3, 3e-4",
                threshold_crumb_64=64,   # compat
                threshold_crumb=64,      # el que usa demo_modified.py
                chunk_size=250,
                output_name=file_dir,
                do_txt=1,
                do_eval=1,
                slices="-1",
            )
            mAP_calculation(args)

            # 5) Leer resultados desde el txt que genera mAP_3Dvolume
            map_txt = os.path.join(file_dir, "nuclu_mAP.txt")
            if os.path.isfile(map_txt):
                with open(map_txt, "r") as f:
                    for line in f:
                        if "mAP @ IoU 0.50" in line:
                            try: mAP_50_total += float(line.split()[-1])
                            except: pass
                        if "mAP @ IoU 0.75" in line:
                            try: mAP_75_total += float(line.split()[-1])
                            except: pass

    # -------- Matching por imagen --------
    if matching_stats:
        try:
            r_stats = matching(mask, img, thresh=matching_stats_ths, report_matches=False)
            all_matching_stats.append(r_stats)
        except Exception as e:
            print(f"[Matching] Error en {id_}: {e}")

    # -------- Matching segCompare por imagen (opcional) --------
    if matching_segcompare:
        if np.max(mask) == 0:
            if verbose: print("GT vacía; se omite segCompare.")
        else:
            try:
                r_stats_segCompare = match_using_segCompare(mask, img)
                all_matching_stats_segCompare.append(r_stats_segCompare)
            except ZeroDivisionError:
                print(f"[segCompare] División por cero en {id_}; se omite.")
            except Exception as e:
                print(f"[segCompare] Error en {id_}: {e}")

# ============ RESUMEN GLOBAL / WRAPPERS ============
print("\n######### RESULTADOS (RESUMEN) #########")

# IoU promedio
iou_fg_mean = float(np.nanmean(iou_fg_list)) if len(iou_fg_list) else None
iou_ov_mean = float(np.nanmean(iou_overall_list)) if len(iou_overall_list) else None

if iou:
    print(f"Foreground IoU medio: {iou_fg_mean}")
    print(f"Overall IoU medio:    {iou_ov_mean}")

# Matching dataset (wrapper)
stats_dataset = None
if matching_stats and len(all_matching_stats) > 0:
    stats_dataset = wrapper_matching_dataset_lazy(all_matching_stats, matching_stats_ths)
    print("Resumen Matching stats (dataset):", stats_dataset)

# Matching segCompare dataset (wrapper)
stats_segCompare_dataset = None
if matching_segcompare and len(all_matching_stats_segCompare) > 0:
    stats_segCompare_dataset = wrapper_matching_segCompare(all_matching_stats_segCompare)
    print("Resumen Matching segCompare (dataset):", stats_segCompare_dataset)

# ============ CREAR CSV **SOLO RESUMEN** ============
# Elegimos, si existe, el DatasetMatching del umbral 0.5
dm_pick = {}
if isinstance(stats_dataset, (list, tuple)) and len(stats_dataset) > 0:
    # intentar coger thresh=0.5
    for dm in stats_dataset:
        d = _datasetmatching_to_dict(dm)
        if d.get("thresh") is not None and abs(d["thresh"] - 0.5) < 1e-6:
            dm_pick = d
            break
    if not dm_pick:
        dm_pick = _datasetmatching_to_dict(stats_dataset[0])

# campos del segCompare 
segc = stats_segCompare_dataset if isinstance(stats_segCompare_dataset, dict) else {}

summary = {
    "foreground_iou_mean": iou_fg_mean,
    "overall_iou_mean":    iou_ov_mean,
    "AP@0.50":             float(mAP_50_total) if mAP else None,
    "AP@0.75":             float(mAP_75_total) if mAP else None,
    "precision":           dm_pick.get("precision"),
    "recall":              dm_pick.get("recall"),
    "accuracy":            dm_pick.get("accuracy"),
    "f1":                  dm_pick.get("f1"),
    "panoptic_quality":    dm_pick.get("panoptic_quality"),
    "mean_true_score":     dm_pick.get("mean_true_score"),
    "mean_matched_score":  dm_pick.get("mean_matched_score"),
    "n_cells":                 float(segc.get("number_of_cells", np.nan)) if segc else None,
    "correct_segmentations":   float(segc.get("correct_segmentations", np.nan)) if segc else None,
    "oversegmentation_rate":   float(segc.get("oversegmentation_rate", np.nan)) if segc else None,
    "undersegmentation_rate":  float(segc.get("undersegmentation_rate", np.nan)) if segc else None,
    "missing_rate":            float(segc.get("missing_rate", np.nan)) if segc else None,
}

# orden de columnas
cols_order = [
    "foreground_iou_mean","overall_iou_mean",
    "AP@0.50","AP@0.75",
    "precision","recall","accuracy","f1","panoptic_quality",
    "mean_true_score","mean_matched_score",
    "n_cells","correct_segmentations",
    "oversegmentation_rate","undersegmentation_rate","missing_rate"
]

df_summary = pd.DataFrame([summary])
df_summary = df_summary[[c for c in cols_order if c in df_summary.columns]]

csv_path = csv_out_dir / f"metrics_{run_name}.csv"
df_summary.to_csv(csv_path, index=False)
print(f"\nCSV RESUMEN guardado en:\n{csv_path}")


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_4.tiff
Foreground IoU: 0.8796726749992244  -- Overall IoU: 0.9058926096765472
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


50it [00:00, 216.53it/s]


	-RUNTIME:	1.008 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.576
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.941
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.801
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_5.tiff
Foreground IoU: 0.7869089321205077  -- Overall IoU: 0.8260354598879411
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


48it [00:00, 213.32it/s]


	-RUNTIME:	0.996 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.901
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.138
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series002_7.tiff
Foreground IoU: 0.7948347301023324  -- Overall IoU: 0.8388822972174783
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


...

	-RUNTIME:	1.174 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.357
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.960
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240920_sdk_Dlg568001-Series001_8.tiff
Foreground IoU: 0.7701124051924694  -- Overall IoU: 0.832676360410654
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


60it [00:00, 261.13it/s]


	-RUNTIME:	1.058 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.335
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.922
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240920_sdk_Dlg568001-Series002_10.tiff
Foreground IoU: 0.8929919763645493  -- Overall IoU: 0.9288535788658747
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


63it [00:00, 241.51it/s]


	-RUNTIME:	1.232 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.871
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


######### RESULTADOS (RESUMEN) #########
Foreground IoU medio: 0.7948695935218023
Overall IoU medio:    0.8426569085738774
Resumen Matching stats (dataset): (DatasetMatching(criterion='iou', thresh=0.3, fp=22, tp=938, fn=57, precision=0.9770833333333333, recall=0.942713567839196, accuracy=0.9223205506391348, f1=0.9595907928388747, n_true=995, n_pred=960, mean_true_score=0.6755890132194787, mean_matched_score=0.7166429298010462, panoptic_quality=0.68768395719016, by_image=False), DatasetMatching(criterion='iou', thresh=0.5, fp=46, tp=914, fn=81, precision=0.9520833333333333, recal

In [ ]:
Cálculo métricas BiaPy + PlantSeg con fake + real (mixed)

In [1]:
# ============================================================
# Calculate_metrics_3D_JGF  — versión con exportación a CSV (RESUMEN SOLO)
# ============================================================

# ---------- CONFIGURACIÓN  ----------
# carpeta EM_Image_Segmentation
code_dir      = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation"
# carpeta mAP_3Dvolume
map_code_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/mAP_3Dvolume"

# carpeta con .tif/.tiff de predicción (Cellpose/PlantSeg/BiaPy...)
# BiaPy + PlantSeg con fake + real (mixed):
input_dir = "C:/Users/Usuario/Desktop/TFM/TFMv2/AllTrainings/fakeAndRealEmbryosTraining/PlantSegInstances"

# carpeta con las GT (mismo nombre que la predicción; si predicción acaba en .tiff, GT puede ser .tif)
gt_dir        = "C:/Users/Usuario/Desktop/TFM/TFMv2/ResUnet 3D GoldStandard - BiaPy/Test/y"
# carpeta temporal donde se generarán ficheros h5/txt
temporal_dir  = "C:/Users/Usuario/Desktop/TFM/TFMv2/Calculate 3D metrics/calculate_3D_metrics/EM_Image_Segmentation/utils/scripts/temp"
# ----------------------------------------------------------

# ---------- OPCIONES DE CÁLCULO ----------
iou                 = True   # calcular IoU y VOC
mAP                 = True   # calcular mAP@0.50 y mAP@0.75
matching_stats      = True   # matching simple (DatasetMatching)
matching_segcompare = True   # matching con segCompare (si está disponible)
matching_stats_ths  = (0.3, 0.5, 0.75)  # umbrales para resumen de matching
verbose             = True
# -----------------------------------------

# ============ IMPORTS BÁSICOS ============
import os, sys, re
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.io import imread
import h5py
from dataclasses import asdict, is_dataclass

# Preparar paths
os.makedirs(temporal_dir, exist_ok=True)
gt_partial_files_dir = os.path.join(temporal_dir, "GT")
os.makedirs(gt_partial_files_dir, exist_ok=True)

# Para poder importar utilidades del repo
sys.path.insert(0, code_dir)
sys.path.insert(0, map_code_dir)

# === Imports de tus utilidades ===
# EM_Image_Segmentation
from utils.matching import matching, match_using_segCompare
from utils.util import save_tif_pair_discard, wrapper_matching_dataset_lazy, wrapper_matching_segCompare
from engine.metrics import jaccard_index_numpy, voc_calculation

# mAP_3Dvolume
from demo_modified import main as mAP_calculation

# ===== Helpers para extraer métricas de DatasetMatching =====
def _datasetmatching_to_dict(dm):
    """
    Intenta leer atributos (precision, recall, accuracy, f1, panoptic_quality,
    mean_true_score, mean_matched_score, thresh). Si no existen (p.ej. es una
    clase distinta), hace un parseo robusto del str(dm).
    """
    keys = ["precision","recall","accuracy","f1","panoptic_quality",
            "mean_true_score","mean_matched_score","thresh"]
    out = {k: None for k in keys}

    # 1) intento directo por atributos
    for k in keys:
        if hasattr(dm, k):
            try:
                out[k] = float(getattr(dm, k))
            except Exception:
                pass

    # 2) si faltan, intento por dataclass
    if any(v is None for v in out.values()) and is_dataclass(dm):
        try:
            d = asdict(dm)
            for k in keys:
                if k in d and out[k] is None:
                    out[k] = float(d[k])
        except Exception:
            pass

    # 3) parseo de str como último recurso
    if any(v is None for v in out.values()):
        s = str(dm)
        for k in keys:
            if out[k] is None:
                m = re.search(rf"{k}\s*=\s*([0-9]*\.?[0-9]+)", s)
                if m:
                    out[k] = float(m.group(1))

    return out

# ============ ACUMULADORES Y CSV ============
run_name   = Path(input_dir.rstrip("/\\")).name
csv_out_dir = Path(temporal_dir) / f"metrics_{run_name}"
csv_out_dir.mkdir(parents=True, exist_ok=True)

iou_fg_list = []
iou_overall_list = []
mAP_50_total = 0.0
mAP_75_total = 0.0
all_matching_stats = []
all_matching_stats_segCompare = []

# ============ LISTADO DE IMÁGENES ============
ids = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.tif', '.tiff'))])
if len(ids) == 0:
    raise RuntimeError(f"No se han encontrado .tif/.tiff en: {input_dir}")

# ============ BUCLE PRINCIPAL POR IMAGEN ============
for id_ in ids:
    # Rutas de predicción y GT
    img_path = os.path.join(input_dir, id_)

    base_noext = os.path.splitext(id_)[0]
    # Si la predicción es .tiff, probamos GT .tif
    if id_.lower().endswith(".tiff"):
        mask_id = base_noext + ".tif"
    else:
        mask_id = id_

    mask_path = os.path.join(gt_dir, mask_id)
    if not os.path.isfile(mask_path):
        # Intenta la otra extensión por si acaso
        alt = base_noext + (".tiff" if id_.lower().endswith(".tif") else ".tif")
        alt_path = os.path.join(gt_dir, alt)
        if os.path.isfile(alt_path):
            mask_path = alt_path
        else:
            print(f"[AVISO] No existe GT para {id_}. Se omite esta imagen.")
            continue

    # Cargar predicción y GT
    img  = imread(img_path).astype(np.int64)
    mask = imread(mask_path).astype(np.int64)

    if verbose:
        print(f"\n### Analizando: {id_}")

    # Carpeta temporal por imagen
    file_dir = os.path.join(temporal_dir, base_noext)
    os.makedirs(file_dir, exist_ok=True)

    # -------- IoU / VOC --------
    if iou:
        _iou    = jaccard_index_numpy((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8))
        _ov_iou = voc_calculation((mask > 0.5).astype(np.uint8), (img > 0.5).astype(np.uint8), _iou)
        iou_fg_list.append(float(_iou))
        iou_overall_list.append(float(_ov_iou))
        if verbose:
            print(f"Foreground IoU: { _iou }  -- Overall IoU: { _ov_iou }")

    # -------- mAP (vía mAP_3Dvolume) --------
    if mAP:
        # 1) H5 de predicción
        h5file_name = os.path.join(file_dir, base_noext + ".h5")
        if os.path.exists(h5file_name):
            try:
                os.remove(h5file_name)
            except PermissionError:
                pass
        with h5py.File(h5file_name, "w") as h5f:
            h5f.create_dataset("dataset", data=img, compression="lzf")

        # 2) H5 de GT (si no existe ya)
        gt_f = os.path.join(gt_partial_files_dir, base_noext + ".h5")
        if not os.path.isfile(gt_f):
            with h5py.File(gt_f, "w") as h5f:
                h5f.create_dataset("dataset", data=mask.squeeze(), compression="lzf")

        # 3) Scores falsos (etiquetas únicas != 0)
        labels = np.unique(img)
        labels = labels[labels != 0].astype(int)

        if labels.size == 0:
            if verbose:
                print(f"[mAP] {id_} no tiene instancias segmentadas. Se omite mAP.")
        else:
            scores = np.ones_like(labels, dtype=np.float32)
            df_scores = pd.DataFrame(np.stack([labels, scores], axis=1))
            score_path = os.path.splitext(h5file_name)[0] + "_scores.txt"
            df_scores.to_csv(score_path, sep="\t", index=False, header=False)

            # --- construir argumentos correctos para mAP_3Dvolume ---
            from types import SimpleNamespace as Namespace
            args = Namespace(
                gt_seg=gt_f,
                predict_seg=h5file_name,
                predict_score=score_path,
                threshold="5e-3, 3e-4",
                threshold_crumb_64=64,   # compat
                threshold_crumb=64,      # el que usa demo_modified.py
                chunk_size=250,
                output_name=file_dir,
                do_txt=1,
                do_eval=1,
                slices="-1",
            )
            mAP_calculation(args)

            # 5) Leer resultados desde el txt que genera mAP_3Dvolume
            map_txt = os.path.join(file_dir, "nuclu_mAP.txt")
            if os.path.isfile(map_txt):
                with open(map_txt, "r") as f:
                    for line in f:
                        if "mAP @ IoU 0.50" in line:
                            try: mAP_50_total += float(line.split()[-1])
                            except: pass
                        if "mAP @ IoU 0.75" in line:
                            try: mAP_75_total += float(line.split()[-1])
                            except: pass

    # -------- Matching por imagen --------
    if matching_stats:
        try:
            r_stats = matching(mask, img, thresh=matching_stats_ths, report_matches=False)
            all_matching_stats.append(r_stats)
        except Exception as e:
            print(f"[Matching] Error en {id_}: {e}")

    # -------- Matching segCompare por imagen (opcional) --------
    if matching_segcompare:
        if np.max(mask) == 0:
            if verbose: print("GT vacía; se omite segCompare.")
        else:
            try:
                r_stats_segCompare = match_using_segCompare(mask, img)
                all_matching_stats_segCompare.append(r_stats_segCompare)
            except ZeroDivisionError:
                print(f"[segCompare] División por cero en {id_}; se omite.")
            except Exception as e:
                print(f"[segCompare] Error en {id_}: {e}")

# ============ RESUMEN GLOBAL / WRAPPERS ============
print("\n######### RESULTADOS (RESUMEN) #########")

# IoU promedio
iou_fg_mean = float(np.nanmean(iou_fg_list)) if len(iou_fg_list) else None
iou_ov_mean = float(np.nanmean(iou_overall_list)) if len(iou_overall_list) else None

if iou:
    print(f"Foreground IoU medio: {iou_fg_mean}")
    print(f"Overall IoU medio:    {iou_ov_mean}")

# Matching dataset (wrapper)
stats_dataset = None
if matching_stats and len(all_matching_stats) > 0:
    stats_dataset = wrapper_matching_dataset_lazy(all_matching_stats, matching_stats_ths)
    print("Resumen Matching stats (dataset):", stats_dataset)

# Matching segCompare dataset (wrapper)
stats_segCompare_dataset = None
if matching_segcompare and len(all_matching_stats_segCompare) > 0:
    stats_segCompare_dataset = wrapper_matching_segCompare(all_matching_stats_segCompare)
    print("Resumen Matching segCompare (dataset):", stats_segCompare_dataset)

# ============ CREAR CSV **SOLO RESUMEN** ============
# Elegimos, si existe, el DatasetMatching del umbral 0.5
dm_pick = {}
if isinstance(stats_dataset, (list, tuple)) and len(stats_dataset) > 0:
    # intentar coger thresh=0.5
    for dm in stats_dataset:
        d = _datasetmatching_to_dict(dm)
        if d.get("thresh") is not None and abs(d["thresh"] - 0.5) < 1e-6:
            dm_pick = d
            break
    if not dm_pick:
        dm_pick = _datasetmatching_to_dict(stats_dataset[0])

# campos del segCompare 
segc = stats_segCompare_dataset if isinstance(stats_segCompare_dataset, dict) else {}

summary = {
    "foreground_iou_mean": iou_fg_mean,
    "overall_iou_mean":    iou_ov_mean,
    "AP@0.50":             float(mAP_50_total) if mAP else None,
    "AP@0.75":             float(mAP_75_total) if mAP else None,
    "precision":           dm_pick.get("precision"),
    "recall":              dm_pick.get("recall"),
    "accuracy":            dm_pick.get("accuracy"),
    "f1":                  dm_pick.get("f1"),
    "panoptic_quality":    dm_pick.get("panoptic_quality"),
    "mean_true_score":     dm_pick.get("mean_true_score"),
    "mean_matched_score":  dm_pick.get("mean_matched_score"),
    "n_cells":                 float(segc.get("number_of_cells", np.nan)) if segc else None,
    "correct_segmentations":   float(segc.get("correct_segmentations", np.nan)) if segc else None,
    "oversegmentation_rate":   float(segc.get("oversegmentation_rate", np.nan)) if segc else None,
    "undersegmentation_rate":  float(segc.get("undersegmentation_rate", np.nan)) if segc else None,
    "missing_rate":            float(segc.get("missing_rate", np.nan)) if segc else None,
}

# orden de columnas
cols_order = [
    "foreground_iou_mean","overall_iou_mean",
    "AP@0.50","AP@0.75",
    "precision","recall","accuracy","f1","panoptic_quality",
    "mean_true_score","mean_matched_score",
    "n_cells","correct_segmentations",
    "oversegmentation_rate","undersegmentation_rate","missing_rate"
]

df_summary = pd.DataFrame([summary])
df_summary = df_summary[[c for c in cols_order if c in df_summary.columns]]

csv_path = csv_out_dir / f"metrics_{run_name}.csv"
df_summary.to_csv(csv_path, index=False)
print(f"\nCSV RESUMEN guardado en:\n{csv_path}")


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_4.tiff
Foreground IoU: 0.8012256215432683  -- Overall IoU: 0.8313896187457553
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


53it [00:00, 116.33it/s]


	-RUNTIME:	1.6 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.445
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.951
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.134
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series001_5.tiff
Foreground IoU: 0.9130827242595183  -- Overall IoU: 0.9234821930923585
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


51it [00:00, 152.19it/s]


	-RUNTIME:	1.296 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.631
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.960
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.781
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240903_MatWRi_Ed633_Dlg568008_Series002_7.tiff
Foreground IoU: 0.9175867129098005  -- Overall IoU: 0.932791925041514
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


...

	-RUNTIME:	1.253 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.665
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.951
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.873
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


### Analizando: 20240920_sdk_Dlg568001-Series001_8.tiff
Foreground IoU: 0.920463888489246  -- Overall IoU: 0.940692400801071
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


60it [00:00, 202.17it/s]


	-RUNTIME:	1.333 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.566
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.950
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.619
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.002


### Analizando: 20240920_sdk_Dlg568001-Series002_10.tiff
Foreground IoU: 0.846274319666826  -- Overall IoU: 0.8935861742231661
	1. Load data
		 Load prediction score
	2. Compute IoU
	 compute bounding boxes
		 chunk 0
	 compute iou matching


78it [00:00, 101.31it/s]


	-RUNTIME:	1.883 [sec]

start evaluation
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.232
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.483
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.245
 Average Precision  (AP) @[ IoU=0.90      | area=   all | maxDets=100 ] = 0.000


######### RESULTADOS (RESUMEN) #########
Foreground IoU medio: 0.8694945860659296
Overall IoU medio:    0.9001529335873144
Resumen Matching stats (dataset): (DatasetMatching(criterion='iou', thresh=0.3, fp=51, tp=941, fn=54, precision=0.9485887096774194, recall=0.9457286432160804, accuracy=0.8996175908221797, f1=0.9471565173628586, n_true=995, n_pred=992, mean_true_score=0.7432492979806872, mean_matched_score=0.7859012236883992, panoptic_quality=0.7443714660199131, by_image=False), DatasetMatching(criterion='iou', thresh=0.5, fp=85, tp=907, fn=88, precision=0.9143145161290323, re